In [16]:
import pandas as pd
import geopandas as gpd
import re
import nltk
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec
import pickle

In [17]:
GIST = gpd.read_parquet("../data/large_files/GIST_geocontexts.parquet")

In [18]:
GIST.sample(10)[["PHI_ID", "clean_text_interpretive_word", "lemmata", "raw_date", "not_before", "not_after", "random_dates","geometry"]]

,PHI_ID,clean_text_interpretive_word,lemmata,raw_date,not_before,not_after,random_dates,geometry
74302,226583,τὸ προσκύνημα Ἀκολούθου Πετεήσιου καὶ τοῦ υἱοῦ...,"{'data': ['προσκύνημα', 'ἀκόλουθος', 'πετεήσιο...",Latopolis (Isna),NaN,NaN,None,POINT (32.54545 25.27350)
172384,344825,ΗΝΔΙΟ εὐχὴν δακρ Υ,"{'data': ['ηνδιο', 'εὐχή', 'δακρ', 'υ']}",2nd c. BC,-200.0,-101.0,"[-151, -164, -149, -177, -156, -137, -134, -12...",POINT (26.94260 37.68994)
138015,303543,Ἡραμνέου Ἐπιφανίου,"{'data': ['ἡραμνέου', 'ἐπιφανίου']}",Hellenistic period,-330.0,-30.0,"[-117, -237, -264, -277, -43, -119, -126, -161...",POINT (33.73650 35.39943)
173575,346267,ἱερεὺς Μᾶρκος Αὐρήλιος Οὐαριανὸς Μειδιανὸς Περ...,"{'data': ['ἱερεύς', 'Μᾶρκος', 'οὐαριανὸς', 'με...",ca. 226 AD,221.0,231.0,"[223, 228, 223, 228, 222, 226, 229, 227, 222, ...",POINT (30.46720 36.98300)
70710,222760,Ἁθὺρ δʹ λόγος δραχμῶν Σφανέμων Σακαούειν αʹ πε...,"{'data': ['ἁθὺρ', 'λόγος', 'δραχμή', 'σφανέμων...",2nd c. AD,101.0,200.0,"[113, 196, 195, 137, 125, 149, 116, 199, 157, ...",POINT (26.41667 28.91667)
57346,200295,ΜΙ ΠΑΣΧΟ ΤΙΑΣ,{'data': ['πασχο']},1st c. BC,-100.0,-1.0,"[-31, -64, -74, -75, -18, -27, -96, -22, -53, ...",POINT (25.73875 35.00953)
93989,248659,ονα,{'data': ['ονα']},IEph 1519f.2,NaN,NaN,None,POINT (27.34551 37.94218)
101199,255954,ἐπικληρῶσαι αὐτὸν καὶ εἰς φυλὴν καὶ εἰς χιλιασ...,"{'data': ['ἐπικληρόω', 'φυλή', 'χιλιαστύν', 'λ...","JÖAI 59,1989,189/90, Nr. 22",NaN,NaN,None,POINT (27.34551 37.94218)
126494,289701,καὶ τήνδ’ Ἀργείην Διὸς αἰγιόχοιο σύνευνον εἵσα...,"{'data': ['τήνδ', 'Ἀργεία', 'Ζεύς', 'αἰγίοχος'...","AM 29,1904,280 unten Nr.3",NaN,NaN,None,POINT (27.91065 39.60672)
134685,300039,MVI V,"{'data': ['mvi', 'v']}",2nd/3rd c. AD,101.0,300.0,"[143, 120, 110, 157, 225, 130, 222, 185, 290, ...",POINT (25.60000 45.81667)


In [19]:
GIST

,PHI_ID,reference,metadata,lines,text_raw,text_iphi,clean_text_conservative,clean_text_interpretive_word,lemmata,raw_date,...,y_lat,TMgeo_name,geometry,name,polis_geo,polis_dist,polis?,duration,random_dates,greece?
0,1,IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,12.0,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,εδοχσεν τοι δεμοι τ[ος ε σ]αλαμ[ινι κλεροχ]ος ...,ἔδοχσεν το͂ι δέμοι ταλαμος οἰκε͂ν ἐᾶ Σαλαμῖνι ...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,"{'data': ['ἔδοχσεν', 'δέμοι', 'Σαλαμίς', 'κλερ...",c. 510-500 a.,...,37.972747,Athenai,POINT (23.72399 37.97275),361-Athenai,"[23.7278, 37.9778]",653.43,True,13.0,"[-510, -504, -510, -507, -506, -500, -508, -50...",True
1,4,IG I³\n4,Att. — stoich. 38 — 485/4 a.,56.0,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,[--------------------------------------] [----...,δέ τις αν ἒ φρορὰν μ ντέκοντα δχμὰς τ ας ℎες π...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,"{'data': ['τὶς', 'φρορὰν', 'πεντέκοντα', 'δραχ...",485/4 a.,...,37.972747,Athenai,POINT (23.72399 37.97275),361-Athenai,"[23.7278, 37.9778]",653.43,True,2.0,"[-485, -485, -485, -484, -485, -484, -485, -48...",True
2,5,IG I³\n5,Att. — c. 500 a.,6.0,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,[εδοχσε]ν [ τει βολει] και [τ]οι δεμοι οτε παρ...,ν καὶ ο͂ι δέμοι ℎότε Παραιβάτες λεια θν τὸς ℎι...,ἔδοχσεν τε͂ι βολε͂ι καὶ το͂ι δέμοι ℎότε Παραιβ...,"{'data': ['ἔδοχσεν', 'τει', 'βολει', 'δέμοι', ...",c. 500 a.,...,38.044135,Eleusis,POINT (23.54150 38.04414),362-Eleusis,"[23.5423, 38.0394]",530.31,True,11.0,"[-498, -502, -504, -503, -496, -504, -496, -49...",True
3,6,IG I³\n6,Att. — stoich. 23/11 — ante 460 a.,160.0,— — — — — — — — — — — — —\n[․․․․․․15․․․․․․․] δ...,------------- [---------------] δραχμεισ[ι ---...,δραχμε͂ιστες τὸς ιο μενος δεμο το͂ν πόλεο οκε͂...,δραχμε͂ισι τες τὸς ιο μενος δεμο το͂ν πόλεον δ...,"{'data': ['δραχμεισι', 'τες', 'μένος', 'δεμο',...",ante 460 a.,...,38.044135,Eleusis,POINT (23.54150 38.04414),362-Eleusis,"[23.5423, 38.0394]",530.31,True,NaN,"[-461, -461, -461, -461, -461, -461, -461, -46...",True
4,11,IG I³\n11,Att. — stoich. 48 — ante med. s. V a.,22.0,[χσυμμαχία καὶ ℎόρ]κ̣ο̣[ς] Ἀ̣[θ]ε̣ν̣α̣[ίον κα]...,[χσυμμαχια και ορ]κο[ς] α[θ]ενα[ιον κα]ι εγεστ...,κο Ἀεναὶ Ἐγεσταί ο͂ι ἐπρυτάνευε οΙ ον ἐ͂ρχε Ἀ...,χσυμμαχία καὶ ℎόρκος Ἀθεναίον καὶ Ἐγεσταίον ἔδ...,"{'data': ['χσυμμαχία', 'ὅρκος', 'ἀθεναίον', 'ἐ...",ante med. s. V a.,...,37.972747,Athenai,POINT (23.72399 37.97275),361-Athenai,"[23.7278, 37.9778]",653.43,True,NaN,"[-456, -456, -456, -456, -456, -456, -456, -45...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180056,353786,IK Strat. III\nAdd. et Corr. 645,Caria — Lagina — Rom. Imp. period? — IK Strato...,2.0,[Ἀρισ]τείδης Ἱεροκλέους τοῦ Θευδώρου [— —]\n,[αρισ]τειδης ιεροκλεους του θευδωρου [--].,τείδης Ἱεροκλέους τοῦ Θευδώρου,Ἀριστείδης Ἱεροκλέους τοῦ Θευδώρου,"{'data': ['ἀριστείδης', 'ἱεροκλέους', 'θευδώρο...",Rom. Imp. period?,...,37.377952,Lagina,POINT (28.03856 37.37795),892-Idrias,"[28.16666667, 37.25]",18182.66,False,441.0,"[335, 308, 402, 205, 178, 158, 26, 387, 390, 3...",False
180057,353787,IK Strat. III\nAdd. et Corr. 712,Caria — Lagina — Rom. Imp. period? — IK Strato...,3.0,[Ἄπφιον] Ἀρτεμιδώ-\n[ρου] ...\n,[απφιον] αρτεμιδω[ρου].,Ἀρτεμιδώ,Ἄπφιον Ἀρτεμιδώρου,"{'data': ['ἄπφιον', 'ἀρτεμιδώρου']}",Rom. Imp. period?,...,37.377952,Lagina,POINT (28.03856 37.37795),892-Idrias,"[28.16666667, 37.25]",18182.66,False,441.0,"[136, -10, 218, 378, 193, 60, 50, 320, 235, 15...",False
180058,353788,IK Strat. III\nAdd. et Corr. 809,Caria — Stratonikeia — Yeşilbağcılar — Helleni...,2.0,καὶ κριὸν [— —]\n,και κριον [--].,καὶ κριὸν,καὶ κριὸν,{'data': ['κριός']},Hellenistic period,...,37.313590,Stratonikeia,POINT (28.06450 37.31359),892-Idrias,"[28.16666667, 37.25]",11484.64,False,301.0,"[-117, -48, -51, -120, -232, -42, -230, -222, ...",False
180059,353789,IK Strat. III\nAdd. et Corr. 1002,Caria — Stratonikeia — 276 BC — IK Stratonikei.

In [20]:
GIST = GIST[GIST["random_dates"].notnull()]
len(GIST)

106898

In [21]:
sum(GIST["polis?"])

70503

In [22]:
GIST_lemmata_merged = [[l for l in inscr_lem["data"]] for inscr_lem in GIST["lemmata"].tolist()]
GIST_lemmata_merged = [l for i in GIST_lemmata_merged for l in i]
len(GIST_lemmata_merged)

1648240

In [23]:
freqs_tups = nltk.FreqDist(GIST_lemmata_merged).most_common()
freqs_tups[:100]

[('δῆμος', 14979),
 ('θεός', 13267),
 ('πόλις', 12561),
 ('ἄλλος', 11026),
 ('ἔτος', 9281),
 ('βουλή', 7875),
 ('ἔχω', 6723),
 ('υἱός', 6261),
 ('εἰμί', 6229),
 ('αὐτός', 6069),
 ('ἄρχω', 5996),
 ('μείς', 5943),
 ('δοκέω', 5934),
 ('δίδωμι', 5796),
 ('ποιέω', 5397),
 ('πᾶς', 5332),
 ('χαίρω', 5308),
 ('ἀγαθός', 5307),
 ('οὗτος', 5272),
 ('γίγνομαι', 5045),
 ('ἀνήρ', 4832),
 ('Ἀπόλλων', 4815),
 ('δραχμή', 4526),
 ('ἀνατίθημι', 4472),
 ('ἱερόν', 4217),
 ('βασιλεύς', 4086),
 ('γυνή', 4058),
 ('χάρις', 4042),
 ('Ζεύς', 4009),
 ('ἱερεύς', 3930),
 ('et', 3844),
 ('σεβαστός', 3752),
 ('μέγας', 3492),
 ('αὐτοκράτωρ', 3482),
 ('ἄρχων', 3389),
 ('νόμος', 3369),
 ('Καῖσαρ', 3332),
 ('εὐεργέτης', 3327),
 ('λέγω', 3288),
 ('ἴδιος', 3224),
 ('ἱερός', 3178),
 ('τιμή', 3097),
 ('Διονύσιος', 3087),
 ('ψήφισμα', 3044),
 ('ὁλκή', 3038),
 ('φιάλη', 2893),
 ('Ἀπολλώνιος', 2875),
 ('μήτηρ', 2843),
 ('τὶς', 2838),
 ('πατήρ', 2782),
 ('Ἀθηναῖος', 2757),
 ('χρύσεος', 2757),
 ('στήλη', 2627),
 ('στέφανος', 2591

In [89]:
pickle.dump(freqs_tups, open("../data/freqs_tups.pickle", "wb"))

In [24]:
match_pattern = "(κατα|ὑπό|ἐκ|ἀ|ἄ|ἀντί|^)δ[ι|ί]κ(α|ά|ο|ό|ί|έ|η)(.+)?"
dik_words = [word for word in GIST_lemmata_merged if re.search(match_pattern, word)]
dik_words_df = pd.DataFrame(nltk.FreqDist(dik_words).most_common(), columns=["lemma", "count"])
dik_words_df.head(5)

,lemma,count
0,δίκη,1153
1,δίκαιος,880
2,δικαστής,606
3,δικαιοσύνη,419
4,δικαστήριον,221


In [25]:
dik_words_df[:100]

,lemma,count
0,δίκη,1153
1,δίκαιος,880
2,δικαστής,606
3,δικαιοσύνη,419
4,δικαστήριον,221
...,...,...
95,δικαστάνς,3
96,δικαιοπραγήσαντα,3
97,δικάδδεν,3
98,δικαδδέτο,3


In [26]:
def get_ngrams(lemmata_data):
    lemmata_list = lemmata_data["data"]
    ngrams = []
    try:
        bigrams = list(nltk.ngrams(lemmata_list, 2))
        trigrams = list(nltk.ngrams(lemmata_list, 3, pad_right=True, pad_left=True))
        fivegrams = list(nltk.ngrams(lemmata_list, 5, pad_right=True, pad_left=True))
        ngrams_merged = bigrams + trigrams + fivegrams
        ngrams_merged = [[i for i in ngram if i is not None] for ngram in ngrams_merged]
        ngrams_merged = [ngram for ngram in ngrams_merged if (len(ngram) > 1)]
    except:
        ngrams_merged = [[]]
    return ngrams_merged


In [35]:
print(get_ngrams({"data" : ["lemma1", "lemma2", "lemma3", "lemma4", "lemma5", "lemma6"]}))

[['lemma1', 'lemma2'], ['lemma2', 'lemma3'], ['lemma3', 'lemma4'], ['lemma4', 'lemma5'], ['lemma5', 'lemma6'], ['lemma1', 'lemma2'], ['lemma1', 'lemma2', 'lemma3'], ['lemma2', 'lemma3', 'lemma4'], ['lemma3', 'lemma4', 'lemma5'], ['lemma4', 'lemma5', 'lemma6'], ['lemma5', 'lemma6'], ['lemma1', 'lemma2'], ['lemma1', 'lemma2', 'lemma3'], ['lemma1', 'lemma2', 'lemma3', 'lemma4'], ['lemma1', 'lemma2', 'lemma3', 'lemma4', 'lemma5'], ['lemma2', 'lemma3', 'lemma4', 'lemma5', 'lemma6'], ['lemma3', 'lemma4', 'lemma5', 'lemma6'], ['lemma4', 'lemma5', 'lemma6'], ['lemma5', 'lemma6']]


In [28]:
%%time
GIST["ngrams"] = GIST["lemmata"].apply(get_ngrams)

CPU times: user 4.2 s, sys: 221 ms, total: 4.42 s
Wall time: 4.52 s


/Users/kasev/Projects/EpiMR/epimr_venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
GIST.head(5)

,PHI_ID,reference,metadata,lines,text_raw,text_iphi,clean_text_conservative,clean_text_interpretive_word,lemmata,raw_date,...,TMgeo_name,geometry,name,polis_geo,polis_dist,polis?,duration,random_dates,greece?,ngrams
0,1,IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,12.0,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,εδοχσεν τοι δεμοι τ[ος ε σ]αλαμ[ινι κλεροχ]ος ...,ἔδοχσεν το͂ι δέμοι ταλαμος οἰκε͂ν ἐᾶ Σαλαμῖνι ...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,"{'data': ['ἔδοχσεν', 'δέμοι', 'Σαλαμίς', 'κλερ...",c. 510-500 a.,...,Athenai,POINT (23.72399 37.97275),361-Athenai,"[23.7278, 37.9778]",653.43,True,13.0,"[-510, -504, -510, -507, -506, -500, -508, -50...",True,"[[ἔδοχσεν, δέμοι], [δέμοι, Σαλαμίς], [Σαλαμίς,..."
1,4,IG I³\n4,Att. — stoich. 38 — 485/4 a.,56.0,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,[--------------------------------------] [----...,δέ τις αν ἒ φρορὰν μ ντέκοντα δχμὰς τ ας ℎες π...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,"{'data': ['τὶς', 'φρορὰν', 'πεντέκοντα', 'δραχ...",485/4 a.,...,Athenai,POINT (23.72399 37.97275),361-Athenai,"[23.7278, 37.9778]",653.43,True,2.0,"[-485, -485, -485, -484, -485, -484, -485, -48...",True,"[[τὶς, φρορὰν], [φρορὰν, πεντέκοντα], [πεντέκο..."
2,5,IG I³\n5,Att. — c. 500 a.,6.0,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,[εδοχσε]ν [ τει βολει] και [τ]οι δεμοι οτε παρ...,ν καὶ ο͂ι δέμοι ℎότε Παραιβάτες λεια θν τὸς ℎι...,ἔδοχσεν τε͂ι βολε͂ι καὶ το͂ι δέμοι ℎότε Παραιβ...,"{'data': ['ἔδοχσεν', 'τει', 'βολει', 'δέμοι', ...",c. 500 a.,...,Eleusis,POINT (23.54150 38.04414),362-Eleusis,"[23.5423, 38.0394]",530.31,True,11.0,"[-498, -502, -504, -503, -496, -504, -496, -49...",True,"[[ἔδοχσεν, τει], [τει, βολει], [βολει, δέμοι],..."
3,6,IG I³\n6,Att. — stoich. 23/11 — ante 460 a.,160.0,— — — — — — — — — — — — —\n[․․․․․․15․․․․․․․] δ...,------------- [---------------] δραχμεισ[ι ---...,δραχμε͂ιστες τὸς ιο μενος δεμο το͂ν πόλεο οκε͂...,δραχμε͂ισι τες τὸς ιο μενος δεμο το͂ν πόλεον δ...,"{'data': ['δραχμεισι', 'τες', 'μένος', 'δεμο',...",ante 460 a.,...,Eleusis,POINT (23.54150 38.04414),362-Eleusis,"[23.5423, 38.0394]",530.31,True,NaN,"[-461, -461, -461, -461, -461, -461, -461, -46...",True,"[[δραχμεισι, τες], [τες, μένος], [μένος, δεμο]..."
4,11,IG I³\n11,Att. — stoich. 48 — ante med. s. V a.,22.0,[χσυμμαχία καὶ ℎόρ]κ̣ο̣[ς] Ἀ̣[θ]ε̣ν̣α̣[ίον κα]...,[χσυμμαχια και ορ]κο[ς] α[θ]ενα[ιον κα]ι εγεστ...,κο Ἀεναὶ Ἐγεσταί ο͂ι ἐπρυτάνευε οΙ ον ἐ͂ρχε Ἀ...,χσυμμαχία καὶ ℎόρκος Ἀθεναίον καὶ Ἐγεσταίον ἔδ...,"{'data': ['χσυμμαχία', 'ὅρκος', 'ἀθεναίον', 'ἐ...",ante med. s. V a.,...,Athenai,POINT (23.72399 37.97275),361-Athenai,"[23.7278, 37.9778]",653.43,True,NaN,"[-456, -456, -456, -456, -456, -456, -456, -45...",True,"[[χσυμμαχία, ὅρκος], [ὅρκος, ἀθεναίον], [ἀθενα..."


In [30]:
%%time
f = open("../data/large_files/corpus_ngrams.txt", "w", encoding="utf-8")
line = 0
ids_lines = {}
for n in range(len(GIST)):
    id = str(GIST.iloc[n]["PHI_ID"])
    try:
        ngrams = [" ".join(ngram) for ngram in GIST.iloc[n]["ngrams"]]
        f.writelines("\n".join(ngrams)+"\n")
        ids_lines[id] = (line, line+len(ngrams))
        line += len(ngrams)
    except:
        print(id)

CPU times: user 17 s, sys: 773 ms, total: 17.8 s
Wall time: 18.7 s


In [31]:
pickle.dump(ids_lines, open("../data/ids_lines.pickle", "wb"))

In [32]:
class NgramCorpus:
    def __iter__(self):
        with open("../data/large_files/corpus_ngrams.txt", "r") as f:
            lines = f.readlines()
            for phi_id in ids_list:
                for line in lines[ids_lines[phi_id][0] : ids_lines[phi_id][1]]:
                    yield line.split()

In [33]:
ids_list = ["1","4","15","17"]

In [34]:
corpus = NgramCorpus()

In [30]:
len([doc for doc in corpus])

904

In [31]:
ids_list = GIST[GIST["random_dates"].apply(lambda x: -500 < x[0] < -401)]["PHI_ID"].astype("str").tolist()
len(ids_list)

6604

In [32]:
corpus = NgramCorpus()

In [33]:
corpus_len = len([doc for doc in corpus])
corpus_len

133348

In [34]:
N_tokens = sum([el[1] for el in freqs_tups])
N_tokens

1648240

In [35]:
N_types = len(freqs_tups)
N_types

214400

In [36]:
N_types_covered = 5000
N_tokens_covered = sum([el[1] for el in freqs_tups[:N_types_covered]])
N_tokens_covered / N_tokens

0.658017036353929

In [40]:
model = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=10, workers=8)

In [41]:
model.build_vocab_from_freq(word_freq=dict(freqs_tups[:N_types_covered]))

In [42]:
%%time
model.train(corpus, total_examples=corpus_len, epochs=model.epochs)

CPU times: user 11.4 s, sys: 1.93 s, total: 13.3 s
Wall time: 13.3 s


(2138767, 4125460)

In [43]:
model.wv.most_similar("θεός")

[('λαμπρός', 0.677864670753479),
 ('παρθικοῦ', 0.6593357920646667),
 ('ἔνδοξος', 0.6470581889152527),
 ('αὐτοκράτωρ', 0.6440725922584534),
 ('ἁδριανοῦ', 0.6383450031280518),
 ('μέγας', 0.6350281834602356),
 ('ἐπιφανής', 0.6306490302085876),
 ('ἀρχιερεύς', 0.6289092898368835),
 ('σεβαστός', 0.6269497275352478),
 ('εὐχαριστέω', 0.6178341507911682)]

In [73]:
model = FastText(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8, min_n=4)
model.build_vocab_from_freq(word_freq=dict(freqs_tups[:N_types_covered]))
model.train(corpus, total_examples=corpus_len, epochs=model.epochs)

(3666485, 6064500)

In [76]:
model.wv.most_similar("θεός")

[('τεός', 0.8119252324104309),
 ('ἐτεός', 0.7080932855606079),
 ('σωτήριος', 0.6826785802841187),
 ('σωτήρ', 0.6691166758537292),
 ('παρθένιος', 0.6619709730148315),
 ('εὐσεβείαι', 0.6525436043739319),
 ('θέα', 0.642316460609436),
 ('προσκυνέω', 0.6290892362594604),
 ('ἐλευθέριος', 0.628943920135498),
 ('ἐπιφανής', 0.6255434155464172)]

# Developing century excluding models

In [77]:
for cent in range(-8, 8):
    if cent < 0:
        cent_start, cent_end = cent * 100, cent * 100 + 99
    else:
        cent_start, cent_end = cent * 100 + 1, cent * 100 + 100
    subset = GIST[GIST["random_dates"].apply(lambda x: cent_start < x[0] < cent_end)]
    print(cent_start, cent_end, (len(subset)))


-800 -701 63
-700 -601 382
-600 -501 2623
-500 -401 6604
-400 -301 9668
-300 -201 14232
-200 -101 13448
-100 -1 8443
1 100 9893
101 200 15986
201 300 11913
301 400 4897
401 500 2634
501 600 2271
601 700 421
701 800 126


In [78]:
cent_ids_samples = {}
for cent in range(-8, 6):
    if cent < 0:
        cent_start, cent_end = cent * 100, cent * 100 + 99
    else:
        cent_start, cent_end = cent * 100 + 1, cent * 100 + 100
    subset_ids = GIST[GIST["random_dates"].apply(lambda x: cent_start < x[0] < cent_end)]["PHI_ID"]
    subset_sample_ids = subset_ids.sample(1000, replace=True)
    cent_ids_samples[cent] = subset_sample_ids

In [79]:
# full_model
ids_list = []
for cent in cent_ids_samples.keys():
    ids_list.extend(list(cent_ids_samples[cent]))

In [80]:
len(ids_list)

14000

In [81]:
ids_list = [str(id) for id in ids_list]

In [82]:
corpus = NgramCorpus()

In [83]:
%%time
model = FastText(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8, min_n=3)
model.build_vocab_from_freq(word_freq=dict(freqs_tups[:N_types_covered]))
model.train(corpus, total_examples=corpus_len, epochs=model.epochs)

CPU times: user 23 s, sys: 1.76 s, total: 24.7 s
Wall time: 12.6 s


(4446799, 7471415)

In [84]:
model.wv.most_similar("θεός", topn=100)

[('ἐτεός', 0.8190017342567444),
 ('τεός', 0.8043358325958252),
 ('ἰατρός', 0.7662969827651978),
 ('κολωνός', 0.7587723135948181),
 ('θεαρός', 0.7583391666412354),
 ('σεμνός', 0.7576870322227478),
 ('ἁγνός', 0.7523812055587769),
 ('τρυφερός', 0.7308467030525208),
 ('γαμβρός', 0.7302103638648987),
 ('γεραιός', 0.7300339937210083),
 ('μανός', 0.7294237613677979),
 ('σεισμός', 0.729305624961853),
 ('λυγρός', 0.7234829068183899),
 ('ζοός', 0.7187726497650146),
 ('δημιουργός', 0.7179083824157715),
 ('θοός', 0.7178543210029602),
 ('ληνός', 0.7176640629768372),
 ('ἀγρός', 0.7155147790908813),
 ('ἱλαρός', 0.7150009274482727),
 ('λαός', 0.7125334143638611),
 ('ὀρός', 0.7107135653495789),
 ('θεσμός', 0.7100199460983276),
 ('θιός', 0.7094550132751465),
 ('γεωργός', 0.7089586853981018),
 ('ψυχρός', 0.7078760862350464),
 ('ἑός', 0.7073254585266113),
 ('ἰτός', 0.7069397568702698),
 ('δεινός', 0.7067515254020691),
 ('βροτός', 0.7054709196090698),
 ('γοργός', 0.7036975026130676),
 ('ζωός', 0.7019622325

In [85]:
model.wv.save("../data/large_files/vectors_fasttext/model_full.wv")

In [87]:
for cent_to_exclude in cent_ids_samples.keys():
    ids_list = []
    for key in cent_ids_samples.keys():
        if key != cent_to_exclude:
            ids_list.extend(list(cent_ids_samples[key]))
    ids_list = [str(id) for id in ids_list]
    model = FastText(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8, min_n=3)
    model.build_vocab_from_freq(word_freq=dict(freqs_tups[:N_types_covered]))
    print("going to train model excluding {} century".format(str(cent_to_exclude)))
    model.train(corpus, total_examples=corpus_len, epochs=model.epochs)
    model.wv.save("../data/large_files/vectors_fasttext/model_without_{}.wv".format(str(cent_to_exclude)))

going to train model excluding -8 century
going to train model excluding -7 century
going to train model excluding -6 century
going to train model excluding -5 century
going to train model excluding -4 century
going to train model excluding -3 century
going to train model excluding -2 century
going to train model excluding -1 century
going to train model excluding 0 century
going to train model excluding 1 century
going to train model excluding 2 century
going to train model excluding 3 century
going to train model excluding 4 century
going to train model excluding 5 century
